<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Entrenamiento de predictor</h4>
</center>

In [1]:
# Importación librerias

# !pip install awswrangler
# !pip install boto3
import boto3
import awswrangler as wr
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")

In [2]:
# Definición fechas

date_start=datetime.strptime('2017-01-01', '%Y-%m-%d')
date_end=datetime.strptime('2019-12-01', '%Y-%m-%d')
forecast_length = 2
date_future=date_end+relativedelta(months=forecast_length)
years=range(2018, 2020)

print(f"Fecha inicio historia {date_start}")
print(f"Fecha fin historia {date_end}")
print(f"Fecha futura (related time series) {date_future}")

timestamp_format="yyyy-MM-dd"
dataset_frequency='M'

# Versión modelo
data_version_cluster1 = '4_c1'
data_version_cluster2 = '4_c2'
data_version_cluster3 = '4_c3'
grouped_cols = ["product_id"]
grouped_cols_name = "product"

Fecha inicio historia 2017-01-01 00:00:00
Fecha fin historia 2019-12-01 00:00:00
Fecha futura (related time series) 2020-02-01 00:00:00


In [3]:
# Conexiones AWS
account_id=boto3.client('sts').get_caller_identity().get('Account')
session = boto3.Session() 
forecast = session.client(service_name='forecast')
region = forecast.meta.region_name
forecastquery = session.client(service_name='forecastquery')
role = f'arn:aws:iam::{account_id}:role/ForecastRole'

In [4]:
# Qué valores predecir: media o percentiles
quantiles=['mean']
# Horizonte de predicción
forecast_length = 2
dataset_group_arn_cluster1=f'arn:aws:forecast:us-east-1:{account_id}:dataset-group/modelo_{data_version_cluster1}_{grouped_cols_name}'
dataset_group_arn_cluster2=f'arn:aws:forecast:us-east-1:{account_id}:dataset-group/modelo_{data_version_cluster2}_{grouped_cols_name}'
dataset_group_arn_cluster3=f'arn:aws:forecast:us-east-1:{account_id}:dataset-group/modelo_{data_version_cluster3}_{grouped_cols_name}'

In [7]:
# Verificar comunicación con Amazon Forecast
assert forecast.list_predictors()

In [8]:
# CLUSTER1
predictor_name_automl_cluster1 = f"modelo_{data_version_cluster1}_predictor"
response = forecast.create_auto_predictor(
    PredictorName=predictor_name_automl_cluster1,
    ForecastHorizon=forecast_length,
    ForecastTypes=quantiles,
    # ForecastDimensions=grouped_cols,
    ForecastFrequency=dataset_frequency,
    DataConfig={
        'DatasetGroupArn': dataset_group_arn_cluster1,
        'AttributeConfigs': [
		{
		"AttributeName": "target_value",
		"Transformations": {
			"aggregation": "sum",
			"backfill": "zero",
			"frontfill": "none",
			"middlefill": "zero"
		}
		}
	]}, 
    TimeAlignmentBoundary={
        'DayOfMonth': 1
    },
    OptimizationMetric='MAPE',
    ExplainPredictor=False
)

In [9]:
automl_predictor_arn_wape = response['PredictorArn']
print(f"Waiting for Predictor with ARN {automl_predictor_arn_wape} to become ACTIVE. \nCurrent Status:")
forecast.describe_auto_predictor(PredictorArn=automl_predictor_arn_wape)['Status']

Waiting for Predictor with ARN arn:aws:forecast:us-east-1:935182042847:predictor/modelo_4_c1_predictor_01J02Q23DBHN6P2C8QM61MAY3F to become ACTIVE. 
Current Status:


'CREATE_IN_PROGRESS'

In [10]:
# CLUSTER2
predictor_name_automl_cluster2 = f"modelo_{data_version_cluster2}_predictor"
response_cluster2 = forecast.create_auto_predictor(
    PredictorName=predictor_name_automl_cluster2,
    ForecastHorizon=forecast_length,
    ForecastTypes=quantiles,
    # ForecastDimensions=grouped_cols,
    ForecastFrequency=dataset_frequency,
    DataConfig={
        'DatasetGroupArn': dataset_group_arn_cluster2,
        'AttributeConfigs': [
		{
		"AttributeName": "target_value",
		"Transformations": {
			"aggregation": "sum",
			"backfill": "zero",
			"frontfill": "none",
			"middlefill": "zero"
		}
		}
	]}, 
    TimeAlignmentBoundary={
        'DayOfMonth': 1
    },
    OptimizationMetric='MAPE',
    ExplainPredictor=False
)

In [11]:
automl_predictor_arn_wape_cluster2 = response_cluster2['PredictorArn']
print(f"Waiting for Predictor with ARN {automl_predictor_arn_wape_cluster2} to become ACTIVE. \nCurrent Status:")
forecast.describe_auto_predictor(PredictorArn=automl_predictor_arn_wape_cluster2)['Status']

Waiting for Predictor with ARN arn:aws:forecast:us-east-1:935182042847:predictor/modelo_4_c2_predictor_01J02Q4FXFQADEHVVGBN5ETE7C to become ACTIVE. 
Current Status:


'CREATE_PENDING'

In [12]:
# CLUSTER3
predictor_name_automl_cluster3 = f"modelo_{data_version_cluster3}_predictor"
response_cluster3 = forecast.create_auto_predictor(
    PredictorName=predictor_name_automl_cluster3,
    ForecastHorizon=forecast_length,
    ForecastTypes=quantiles,
    # ForecastDimensions=grouped_cols,
    ForecastFrequency=dataset_frequency,
    DataConfig={
        'DatasetGroupArn': dataset_group_arn_cluster3,
        'AttributeConfigs': [
		{
		"AttributeName": "target_value",
		"Transformations": {
			"aggregation": "sum",
			"backfill": "zero",
			"frontfill": "none",
			"middlefill": "zero"
		}
		}
	]}, 
    TimeAlignmentBoundary={
        'DayOfMonth': 1
    },
    OptimizationMetric='MAPE',
    ExplainPredictor=False
)

In [13]:
automl_predictor_arn_wape_cluster3 = response_cluster3['PredictorArn']
print(f"Waiting for Predictor with ARN {automl_predictor_arn_wape_cluster3} to become ACTIVE. \nCurrent Status:")
forecast.describe_auto_predictor(PredictorArn=automl_predictor_arn_wape_cluster3)['Status']

Waiting for Predictor with ARN arn:aws:forecast:us-east-1:935182042847:predictor/modelo_4_c3_predictor_01J02Q5EW0HRWSQEVHF4M5B8WP to become ACTIVE. 
Current Status:


'CREATE_PENDING'

In [ ]:
# CHECK

In [31]:
forecast.describe_auto_predictor(PredictorArn=automl_predictor_arn_wape)['Status']

'ACTIVE'

In [34]:
forecast.describe_auto_predictor(PredictorArn=automl_predictor_arn_wape_cluster2)['Status']

'ACTIVE'

In [35]:
forecast.describe_auto_predictor(PredictorArn=automl_predictor_arn_wape_cluster3)['Status']

'ACTIVE'